In [1]:
import numpy as np
import pandas as pd

In [2]:
from trackml.dataset import load_event
import sys
sys.path.insert(0, '../common')

from utils import get_features

In [3]:
path = '../ipynb/input/train_1/event000001000'
hits, cells, particles, truth = load_event(path)

In [4]:
hits = get_features(hits)

In [5]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,x2,y2,z2,r2,r,eta,phi,absZ,z1
0,1,-64.409897,-7.163700,-1502.5,7,2,1,-0.042829,-0.004763,-23.184208,64.807045,1503.896973,-3.837108,-3.030827,1502.5,-23.184208
1,2,-55.336102,0.635342,-1502.5,7,2,1,-0.036804,0.000423,-27.150467,55.339748,1503.518799,-3.994889,3.130112,1502.5,-27.150467
2,3,-83.830498,-1.143010,-1502.5,7,2,1,-0.055707,-0.000760,-17.921406,83.838287,1504.837158,-3.579932,-3.127959,1502.5,-17.921406
3,4,-96.109100,-8.241030,-1502.5,7,2,1,-0.063835,-0.005474,-15.576118,96.461777,1505.593262,-3.439919,-3.056055,1502.5,-15.576118
4,5,-62.673599,-9.371200,-1502.5,7,2,1,-0.041676,-0.006232,-23.709833,63.370335,1503.835815,-3.859459,-2.993168,1502.5,-23.709833


In [6]:
hits_truth = hits.merge(truth, on='hit_id')

In [7]:
hits_truth.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,x2,y2,z2,...,absZ,z1,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
0,1,-64.409897,-7.163700,-1502.5,7,2,1,-0.042829,-0.004763,-23.184208,...,1502.5,-23.184208,0,-64.411598,-7.164120,-1502.5,250710.000000,-149908.000000,-956385.00000,0.000000
1,2,-55.336102,0.635342,-1502.5,7,2,1,-0.036804,0.000423,-27.150467,...,1502.5,-27.150467,22525763437723648,-55.338501,0.630805,-1502.5,-0.570605,0.028390,-15.49220,0.000010
2,3,-83.830498,-1.143010,-1502.5,7,2,1,-0.055707,-0.000760,-17.921406,...,1502.5,-17.921406,0,-83.828003,-1.145580,-1502.5,626295.000000,-169767.000000,-760877.00000,0.000000
3,4,-96.109100,-8.241030,-1502.5,7,2,1,-0.063835,-0.005474,-15.576118,...,1502.5,-15.576118,297237712845406208,-96.122902,-8.230360,-1502.5,-0.225235,-0.050968,-3.70232,0.000008
4,5,-62.673599,-9.371200,-1502.5,7,2,1,-0.041676,-0.006232,-23.709833,...,1502.5,-23.709833,418835796137607168,-62.659401,-9.375040,-1502.5,-0.281806,-0.023487,-6.57318,0.000009


In [8]:
hits_truth['barcode'] = hits_truth['particle_id']
hits_truth['evtid'] = 0

In [19]:
hits4gg = pd.DataFrame(hits_truth[['evtid', 'barcode', 'r', 'eta', 'phi', 'z', 'volume_id', 'layer_id']])

In [10]:
def make_layer_id(hits):
    data = []
    icount = 0
    volumes = hits['volume_id'].unique()
    vol_groups = hits.groupby('volume_id')
    for name, group in vol_groups:
        layer_groups = group.groupby('layer_id')
        for l_name, l_group in layer_groups:
            icount += 1
            data.append([name, l_name, icount])
        
    return pd.DataFrame(data=data, columns=['volume_id', 'layer_id', 'layer'])

In [11]:
layer_idx = make_layer_id(hits4gg)

In [12]:
layer_idx

,volume_id,layer_id,layer
0,7,2,1
1,7,4,2
2,7,6,3
3,7,8,4
4,7,10,5
5,7,12,6
6,7,14,7
7,8,2,8
8,8,4,9
9,8,6,10


In [42]:
layer_idx.to_csv('layer_id.csv')

In [20]:
hits4gg = hits4gg.merge(layer_idx, on=['volume_id', 'layer_id'])

In [27]:
hits_barrel = hits4gg[ (hits4gg['eta'] < 0.1) & (hits4gg['eta'] > -0.1) ]

In [28]:
hits4train = hits_barrel

In [23]:
layers = np.unique(hits4train['layer'].values)
layers

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [15]:
from graph import construct_graph

In [16]:
start_layers = [8, 9, 10, 1, 25, 26, 27, 28, 41]
end_layers = [9, 10, 11, 25, 26, 27, 28, 41, 42]
layer_pairs = np.stack([start_layers, end_layers], axis=1)
layer_pairs

array([[ 8,  9],
       [ 9, 10],
       [10, 11],
       [ 1, 25],
       [25, 26],
       [26, 27],
       [27, 28],
       [28, 41],
       [41, 42]])

In [17]:
phi_slope_max = 0.001
z0_max_inner = 100
z0_max_outer = 50

In [34]:
print("hits: ", hits4train.shape[0])
print('tracks: ', np.unique(hits4train['barcode']).shape[0])

hits:  4048
tracks:  620


In [37]:
from graph import construct_segments
segments = construct_segments(hits4train, layer_pairs,phi_slope_max, z0_max_inner, z0_max_outer)

In [38]:
segments.shape

(17223, 2)

In [39]:
graphs = construct_graph(hits4train, layer_pairs, phi_slope_max, z0_max_inner, z0_max_outer)

In [1]:
from build_data import build_graphs

In [2]:
build_graphs('../ipynb/input/train_1', 'kaggle2')